<a href="https://colab.research.google.com/github/etatc/NCBIminer/blob/main/NCBI_miner_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!pip install bs4
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

     |████████████████████████████████| 911kB 6.4MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/restricted 

In [2]:
import time
import csv
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
import re

In [3]:
DRIVER_PATH = '/usr/bin/chromedriver'
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [4]:
fasta_urls = []
urls = []
genome = []
i=0

In [5]:
driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=chrome_options)
driver.get("https://www.ncbi.nlm.nih.gov/nuccore/?term=txid9103")
#Clicks to the mrna section for the user
Mrna = driver.find_element_by_xpath("""//*[@data-value_id="mrna"]""").click()
only_nucleotides = driver.find_element_by_xpath("""//*[@data-value_id="nuccore"]""").click()
click_the_box = driver.find_element_by_xpath("""//*[@id="pageno"]""").click()
clear_the_box = driver.find_element_by_xpath("""//*[@id="pageno"]""").send_keys(Keys.BACKSPACE)
no_for_box = driver.find_element_by_xpath("""//*[@id="pageno"]""").send_keys("274")
enter_the_no = driver.find_element_by_xpath("""//*[@id="pageno"]""").send_keys(Keys.ENTER)
#scrape the urls
#!!!change the i value to end of Mrna pages 2XXX!!!
totaltime =time.time()

In [6]:
while i < 1:
    sscrape_time = time.time()
    time.sleep(5)
    page = driver.find_element_by_xpath("//html").get_attribute('outerHTML')
    soup = BeautifulSoup(''.join(page), "lxml")
    for link in soup.find_all(id="ReportShortCut6"):
        f = link.get('href')
        fasta_urls.append(f)
        s = re.sub(r'\?report=fasta', '', f)
        urls.append(s)
    next_page = driver.find_element_by_xpath("""//*[@class="active page_link next"]""").click()
    i = i+1
escrape_time = time.time() - sscrape_time
print(escrape_time)
driver.quit()
write_this_csv = []

6.346516370773315


In [9]:
for url in urls:
    sg_time = time.time()
    webpage = 'https://www.ncbi.nlm.nih.gov' + url
    driver = webdriver.Chrome(executable_path=DRIVER_PATH, options=chrome_options)
    driver.get(webpage)
    time.sleep(1)

    page = driver.find_element_by_xpath("//html").get_attribute('outerHTML')
    soup = BeautifulSoup(''.join(page), "lxml")
    stuff = []
    things = []
    for link in soup.find_all("a", attrs={"sfeat": "CDS"}):
        f = link.find_parent('span')
        q = f.contents[3]
        stuff.append(q)
        s = f.contents[7]
        things.append(s)
    mrna = []
    for thing in soup.find_all("span", attrs={"class": "ff_line"}):
        line = thing.contents[0]
        mrna.append(line)
    driver.quit()
    gene = ""
    for index, item in enumerate(mrna):
        fix = re.sub(" ", "", item)
        gene = gene + fix

    what = re.search('/gene=(.*)\n', stuff[0]).group(1)
    gene_name= re.sub(r'\"', "", what)

    h = re.search('  (.*)\n', stuff[0]).group(1)
    cds = re.sub(r'<', " ", h)

    almost = re.search('/product=(.*)\n', stuff[0]).group(1)
    product = re.sub(r'\"', "", almost)

    test1 = re.sub(r'\n*? ', '', things[0])
    test2 = re.sub(r'/translation="', '', test1)
    test3 = re.sub(r'\n', "", test2)
    amino_seq = re.sub(r'"', "", test3)

    start = re.search(' (.*)\.\.', cds).group(1)
    end = re.search('\.\.(.*)', cds).group(1)

    cds_region = gene[int(start) - 1:int(end)]
    promoter = gene[0:int(start) - 1]
    terminator = gene[int(end):len(gene)]

    write_this = [url, gene_name, product, gene, promoter, cds_region, terminator, amino_seq]
    write_this_csv.append(write_this)

    print(write_this)
    driver.quit()
    eg_time = time.time() - sg_time

    print(eg_time)

['/nuccore/XM_031557269.1', 'LOC104914973', 'uncharacterized protein LOC104914973 isoform X1', 'tgccaatattccgttacatgttgctgtgtgacagctgaaagcagaggagcagcctgacaaaatggcatctgacagggaagtgcacatgaaacaatggtgaaaaattgaattccgccatgcagaaaaaatggcaccttgtgacattcactaatgcttgctgaaccaaacagttgatgtgagcatactgaggcagcgagtggttgcgttttggcagtggcaacagcaacttccgctggtgctgatttttatgaacatgacatgcatactcttgttcattgctggtgaatatgaatcactgatggtggtgactatgtcgaaaaagagtgttctatagttgagaatttgatctataaaacagtgttcttctatctattcaagtttctgtggaaataaatagaaggcattacttttggagtgacctacataatacaatttgtatatactctgtgcgaccctagcaaaccaaaaggttggacacccatgccttagcccaaacagaaccatcctatgcttcctcattcggcttcctcattcggctgcttttcatgctgacctcacaaggtttacacaagaagtaaccaggtaccagcgtttttaatacctcatataaaaaacacgcattatccaactcataactacagtcactcgaatcaagatccaagttgtgtttgcgtgaagtcacacctggacctatctaaacaatgcctctggtaacagtatatccttgtgaattaaatccatgagacaggaaggttcccaaataatcacactctgtcatctcatttttctactgtttcactgtccacagtagggacaacaaaccatgttccagatctaaatatcttgtttcactgccactgaaagattaggagccaaactctagcagcctttctgttcttcatac

In [10]:
with open('turkeygenome.csv', 'w', newline='') as f:
    s_time = time.time()
    writer = csv.writer(f)
    for number, row in enumerate(write_this_csv):
        writer.writerow(write_this_csv[number])
e_time = time.time() - s_time
print(e_time)
print("Done")
totale_time = time.time() - totaltime
print(totale_time)

0.0071010589599609375
Done
170.82067108154297
